Gather fit results for feature reinforcement learning model into a dataframe. 

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import pickle 
from scipy import stats
from scipy.special import logsumexp

# Custom dependencies
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()) + '/libraries/')

## 1. Select fit type

In [23]:
fit_type = 'choice'

## 2. Gather fit results in a dataframe

In [24]:
## Set subjects. 
subjects = [23, 26, 27, 28, 29, 30,  
            31, 34, 35, 36, 37, 38, 39, 40,  
            41, 42, 43, 44, 45, 46, 47, 49, 53,  
            54, 55, 57, 60, 61, 62, 63, 67, 68, 69, 70, 71]
n_subj = len(subjects)

## Set model fits path.
if fit_type == 'choice':
    fit_paths = os.getcwd() + '/'

## Gather fit results in a dataframe. 
fit_all = []
for s in np.arange(n_subj):

    fit_subj = []
    
    ## Load fit for this subject.
    subj_id = subjects[s]
    print('subject: ' + str(subj_id))
    fit_path = fit_paths + 'subj' + str(subj_id) + '_MLE_fit.obj'
    fit_file = open(fit_path,'rb')
    fit = pickle.load(fit_file)
    
    n_iter, a, b = fit.results.shape
    n_parms = len(fit.initial_conditions[0,:])
    
    ## Loop through iterations.
    for i in np.arange(n_iter):
        
        fit_iter = []
        
        ## Append subject and iteration number.
        fit_iter.append([subj_id, i+1])
        
        ## Print initial conditions for this iteration. 
        init_conds = fit.initial_conditions[i,:]
        fit_iter.append(init_conds)
        
        ## Average fit parameter values in this iteration.
        fit_params = np.mean(fit.results[i,:,0:n_parms],axis=0)
        fit_iter.append(fit_params)
        
        ## Average likelihood from all games in this iteration. 
        cv_log_lik = np.mean(fit.results[i,:,-1])
        fit_iter.append(cv_log_lik)

        fit_subj.append(np.hstack(fit_iter))
        
    fit_all.append(np.vstack(fit_subj))
     
model_fit_results = pd.DataFrame(np.vstack(fit_all))

if fit_type == 'choice':
    model_fit_results.columns = ['subj',
                                 'iter',
                                 'learning_rate_init', 
                                 'decay_rate_init', 
                                 'beta_value_choice_init', 
                                 'learning_rate', 
                                 'decay_rate', 
                                 'beta_value_choice', 
                                 'log_lik']


model_fit_results.to_csv(fit_paths + 'results.csv')

subject: 23
subject: 26
subject: 27
subject: 28
subject: 29
subject: 30
subject: 31
subject: 34
subject: 35
subject: 36
subject: 37
subject: 38
subject: 39
subject: 40
subject: 41
subject: 42
subject: 43
subject: 44
subject: 45
subject: 46
subject: 47
subject: 49
subject: 53
subject: 54
subject: 55
subject: 57
subject: 60
subject: 61
subject: 62
subject: 63
subject: 67
subject: 68
subject: 69
subject: 70
subject: 71


In [25]:
bf = model_fit_results.loc[model_fit_results.groupby('subj')['log_lik'].transform(max) == model_fit_results['log_lik']].reset_index(drop=True)
bf

,subj,iter,learning_rate_init,decay_rate_init,beta_value_choice_init,learning_rate,decay_rate,beta_value_choice,log_lik
0,23.0,1.0,0.651575,0.865513,2.996307,0.152972,0.293169,9.460261,-10.246525
1,26.0,2.0,0.666808,0.928436,3.995602,0.176199,0.519723,8.501893,-11.607554
2,27.0,2.0,0.605429,0.297098,3.267801,0.167426,0.349633,8.280453,-10.891095
3,28.0,3.0,0.478658,0.819146,4.077130,0.198064,0.454605,7.226126,-11.293843
4,29.0,3.0,0.661105,0.028189,4.480148,0.105742,0.395521,13.236448,-11.877237
5,30.0,4.0,0.306106,0.061935,3.554288,0.200234,0.446688,5.763142,-13.951775
6,31.0,1.0,0.604059,0.755501,3.862629,0.155047,0.293629,10.586924,-10.218610
7,34.0,5.0,0.350557,0.970366,3.090989,0.086646,0.303309,14.964524,-10.992205
8,35.0,3.0,0.112136,0.220816,3.736344,0.112136,0.220816,3.736344,-13.883660
9,36.0,1.0,0.554441,0.350099,3.370860,0.271914,0.339760,5.837662,-11.975613
